In [1]:
#%matplotlib notebook
%matplotlib inline
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, sys
display(HTML("<style>.container { width:100% !important; }</style>"))
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [2]:
# BASELINE
# TFIDF
# SICK_VERBADJNOUN_lemma	0.611	0.091		# TFIDF
# SICK_ADJNOUN_lemma	0.558	0.111			# TFIDF
# SICK_VERBADJNOUN_orth	0.568	0.106		# TFIDF
# SICK_ADJNOUN_orth	0.542	0.119			# TFIDF

In [3]:
def sim_jaccard(a,b):
    a_set = set(a.split())
    b_set = set(b.split())
    return len(a_set & b_set) / float(len(a_set | b_set))

def sim_len(a,b):
    a_len = len(a.split())
    b_len = len(b.split())
    return 1. - abs(a_len-b_len) / float(max(a_len, b_len))

In [5]:
%cd ~/recursive-dependency-embedding/SA/experiments/results
pd.set_option('display.max_colwidth', -1)

import os.path
if os.path.isfile('SICK_test_annotated_mod.txt'): 
    print('load sick_test from "SICK_test_annotated_mod.txt" ...')
    sick_test = pd.DataFrame.from_csv('SICK_test_annotated_mod.txt', sep='\t')
else:
    print('generate sick_test from "SICK_test_annotated_mod.txt" and save to "SICK_test_annotated.txt"...')
    sick_test = pd.DataFrame.from_csv('SICK_test_annotated.txt', sep='\t', header=0)
    sick_test_overlap = pd.DataFrame.from_csv('semeval_overlap_test.tsv', sep='\t', header=0)
    print(sick_test['relatedness_score'].corr(sick_test_overlap['relatedness_score']))
    sick_test = sick_test.reset_index()
    sick_test['sim_jaccard'] = sick_test.apply(lambda row: sim_jaccard(row['sentence_A'], row['sentence_B']), axis=1)
    sick_test['sim_len'] = sick_test.apply(lambda row: sim_len(row['sentence_A'], row['sentence_B']), axis=1)
    sick_test['len_total'] = sick_test.apply(lambda row: len(row['sentence_A'].split() + row['sentence_B'].split()), axis=1)
    #0.62701073200106172

    # add dependency types
    import spacy
    nlp = spacy.load('en')
    for s_v in ['A', 'B']:
        sick_test['sentence_%s_deps' % s_v] = sick_test.apply(lambda row: [token.dep_ for token in nlp(unicode(row['sentence_%s' % s_v]))], axis=1)

    sick_test.to_csv('SICK_test_annotated_mod.txt', sep='\t')
sick_test

/mnt/DATA/DEVELOPING/PycharmProjects/recursive-dependency-embedding/SA/experiments/results
load sick_test from "SICK_test_annotated_mod.txt" ...


,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,sim_jaccard,sim_len,len_total,sentence_A_deps,sentence_B_deps
0,6,There is no boy playing outdoors and there is no man smiling,A group of kids is playing in a yard and an old man is standing in the background,3.300,NEUTRAL,0.181818,0.666667,30,"[u'expl', u'ROOT', u'det', u'attr', u'acl', u'advmod', u'cc', u'expl', u'conj', u'det', u'attr', u'acl']","[u'det', u'nsubj', u'prep', u'pobj', u'aux', u'ROOT', u'prep', u'det', u'pobj', u'cc', u'det', u'amod', u'nsubj', u'aux', u'conj', u'prep', u'det', u'pobj']"
1,7,A group of boys in a yard is playing and a man is standing in the background,The young boys are playing outdoors and the man is smiling nearby,3.700,NEUTRAL,0.300000,0.705882,29,"[u'det', u'nsubj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'aux', u'ROOT', u'cc', u'det', u'nsubj', u'aux', u'conj', u'prep', u'det', u'pobj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'dobj', u'cc', u'det', u'nsubj', u'aux', u'conj', u'advmod']"
2,8,A group of children is playing in the house and there is no man standing in the background,The young boys are playing outdoors and the man is smiling nearby,3.000,NEUTRAL,0.227273,0.666667,30,"[u'det', u'nsubj', u'prep', u'pobj', u'aux', u'ROOT', u'prep', u'det', u'pobj', u'cc', u'expl', u'conj', u'det', u'attr', u'acl', u'prep', u'det', u'pobj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'dobj', u'cc', u'det', u'nsubj', u'aux', u'conj', u'advmod']"
3,10,A brown dog is attacking another animal in front of the tall man in pants,A brown dog is attacking another animal in front of the man in pants,4.900,ENTAILMENT,0.928571,0.933333,29,"[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'amod', u'pobj', u'prep', u'pobj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'prep', u'pobj']"
4,11,A brown dog is attacking another animal in front of the man in pants,A brown dog is helping another animal in front of the man in pants,3.665,NEUTRAL,0.857143,1.000000,28,"[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'prep', u'pobj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'prep', u'pobj']"
5,13,Two dogs are wrestling and hugging,There is no dog wrestling and hugging,3.300,CONTRADICTION,0.300000,0.857143,13,"[u'nummod', u'nsubj', u'aux', u'ROOT', u'cc', u'conj']","[u'expl', u'ROOT', u'det', u'attr', u'acl', u'cc', u'conj']"
6,15,A brown dog is attacking another animal in front of the man in pants,There is no dog wrestling and hugging,2.700,NEUTRAL,0.111111,0.500000,21,"[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'prep', u'pobj']","[u'expl', u'ROOT', u'det', u'attr', u'acl', u'cc', u'conj']"
7,16,Two dogs are wrestling and hugging,A brown dog is attacking another animal in front of the tall man in pants,2.900,NEUTRAL,0.000000,0.400000,21,"[u'nummod', u'nsubj', u'aux', u'ROOT', u'cc', u'conj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'amod', u'pobj', u'prep', u'pobj']"
8,17,Two dogs are wrestling and hugging,A brown dog is helping another animal in front of the man in pants,2.300,NEUTRAL,0.000000,0.428571,20,"[u'nummod', u'nsubj', u'aux', u'ROOT', u'cc', u'conj']","[u'det', u'amod', u'nsubj', u'aux', u'ROOT', u'det', u'dobj', u'prep', u'pobj', u'prep', u'det', u'pobj', u'prep', u'pobj']"
9,19,A person in a black jacket is doing tricks on a motorbike,A man in a black jacket is doing tricks on a motorbike,4.900,ENTAILMENT,0.833333,1.000000,24,"[u'det', u'nsubj', u'prep', u'det', u'amod', u'pobj', u'aux', u'ROOT', u'dobj', u'prep', u'det', u'pobj']","[u'det', u'nsubj', u'prep', u'det', u'amod', u'pobj', u'aux', u'ROOT', u'dobj', u'prep', u'det', u'pobj']"


In [ ]:

sick_test_loaded

In [ ]:
sick_test.corr()

In [ ]:
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_rows', 1000)

results_all = pd.DataFrame.from_csv('scores_all_new.tsv', sep='\t', index_col=['crossfold_id','dev_file_index'])
results_all['dependency available'] = results_all.apply(lambda row: 'NO' if 'blanked' in row.train_data_path else 'YES', axis=1)
results_all['order aware'] = results_all.apply(lambda row: 'YES' if 'LSTM' in row.tree_embedder else 'NO', axis=1)
#results_all = results_all.set_index(['dependency available', 'order aware'])
results_all['pearson_tfidf'] = 0.618864
results_all['mse_tfidf'] = 0.082341
#results_all.set_index(['crossfold_id','dev_file_index'])
results_all[:10]

In [ ]:
res = results_all[['dependency available', 'order aware', 'score_pearson', 'score_mse', 'pearson_tfidf', 'mse_tfidf']].set_index(['dependency available', 'order aware']).rename(columns={'score_pearson': 'pearson', 'score_mse':'mse'})
res[:10]

In [ ]:
# get subsets
#res.xs('NO', level='dependency available').mean()
#res.xs('NO', level='order aware')
#s = res.xs(('NO', 'NO'), level=['order aware', 'dependency available'])
#pd.Series(s['pearson'])
res.groupby('order aware').mean()

In [ ]:
res.groupby('dependency available').mean()

In [ ]:
import subprocess
def print_images(images, path="images"):
    for name in images:
        p = os.path.join(path, name)
        # save figure as svg
        #images[name].savefig('%s.svg' % p)
        images[name].savefig('%s.pdf' % p, format='pdf', bbox_inches='tight')
        # convert for latex
        #subprocess.call(['inkscape','-D','-z','--file=%s.svg' % p, '--export-pdf=%s.pdf' % p, '--export-latex'])
        print('finished %s' % p)

In [ ]:
measures = ['mse', 'pearson']
#parameter = 'dependency available'
parameters = ['dependency available', 'order aware']
settings = [('NO', 'NO'), ('NO', 'YES'), ('YES', 'NO'), ('YES', 'YES')]

In [ ]:
from matplotlib import rcParams

#rcParams['axes.titlepad'] = 20 
#rcParams['axes.titlepad'] = 20 
images = {}
bootstrap=10000
#rcParams.update({'figure.autolayout': True})

figsize = (4.5, 3)
fig_merged, axs = plt.subplots(1,2, figsize=figsize)
res.boxplot(column='mse', figsize=(figsize[0] / 2, figsize[1]), ax=axs[0], bootstrap=bootstrap, notch=True)
axs[0].invert_yaxis()
#axs[0].minorticks_on()
#axs[0].grid(which='minor', linestyle=':', linewidth='0.3', color='black')
res.boxplot(column='pearson', figsize=(figsize[0] / 2, figsize[1]), ax=axs[1], bootstrap=bootstrap, notch=True)
#axs[1].minorticks_on()
#axs[1].grid(which='minor', linestyle=':', linewidth='0.3', color='black')
fig_merged.suptitle('all runs', y=1.01)
plt.subplots_adjust(wspace=0.5)
images['fig_merged'] = fig_merged

y_title = 1.0
y_suptitle = 1.1

#levels = ['order aware', 'dependency available']
#other_level = {levels[0]: levels[1], levels[1]: levels[0]}
other_parameter = {parameters[0]: parameters[1], parameters[1]:parameters[0]}

def param_short(param):
    return ''.join([w[0].upper() for w in param.split()])

for m in measures:
    axs = res.boxplot(column=m, by=parameters, figsize=figsize, bootstrap=bootstrap, notch=True)
    #axs.minorticks_on()
    #axs.grid(which='minor', linestyle=':', linewidth='0.3', color='black')
    plt.title(m, y=y_suptitle)
    fig_sep = axs.get_figure()
    fig_sep.suptitle('')
    if m == 'mse':
        plt.gca().invert_yaxis()
    images['fig_sep_'+m] = fig_sep

for parameter in parameters:
    fig_sep, axs_sep = plt.subplots(1,2, figsize=figsize)
    for i, v in enumerate(['NO', 'YES']):
        
        #res.xs(v, level=parameter).boxplot(column=['pearson'], by=[other_parameter[parameter]], ax=axs_sep[0], bootstrap=bootstrap, notch=True)
        #axs_sep[0].set_title('pearson', y=y_title)
        #axs_sep[0].set_xlabel('')
        res.xs(v, level=parameter).boxplot(column=['mse'], by=[other_parameter[parameter]], ax=axs_sep[i], bootstrap=bootstrap, notch=True)
        axs_sep[i].set_title('%s = %s' % (param_short(parameter), v), y=y_title)
        axs_sep[i].set_xlabel('')
        axs_sep[i].invert_yaxis()
    fig_sep.suptitle('%s (%s) conditioned by \n%s (%s)' % (other_parameter[parameter], param_short(other_parameter[parameter]), parameter, param_short(parameter)), y=y_suptitle + 0.02)
    #plt.subplots_adjust(left=0.2, wspace=0.8, top=0.8)
    plt.subplots_adjust(wspace=0.5)
    images['fig_sep_cond_%s' % other_parameter[parameter].split()[0]] = fig_sep



In [ ]:
print_images(images)

In [ ]:
import numpy
from pandas import read_csv
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
from math import sqrt
import scipy.stats as st

def bootstrap_convidence(values, n_iterations=1000, alpha=0.95):
    #values = res.xs(('NO', 'NO'))['pearson']
    mean_b = np.mean(values)
    std_b = np.std(values)
    n_b_root = sqrt(len(values))
    # configure bootstrap
    #n_iterations = 10000
    n_size = int(len(values) * 0.10)
    # run bootstrap
    stats = list()
    for i in range(n_iterations):
        samples = resample(values, n_samples=n_size)
        t_i = (np.mean(samples) - mean_b) * n_b_root / np.std(samples)
        stats.append(t_i)
    # plot scores
    #pyplot.hist(stats)
    #pyplot.show()
    
    # confidence intervals
    #print(stats)
    #p_lower = ((1.0-alpha)/2.0) * 100
    p_lower = (alpha/2.) * 100
    #p = (alpha / 2.0) * 100
    lower = numpy.percentile(stats, p_lower)
    #lower = st.t._ppf(p_lower, n_iterations-1)
    #print('lower %.2f: %.3f' % (p_lower, lower))
    #p_upper = (alpha+((1.0-alpha)/2.0)) * 100
    p_upper = (1.-alpha/2.) * 100
    #p = (1.0 - alpha / 2.0) * 100
    upper = numpy.percentile(stats, p_upper)
    #upper = st.t._ppf(p_upper, n_iterations-1)
    #print('upper %.1f: %.3f' % (p_upper, upper))
    c_intv = [mean_b + lower * std_b / n_b_root, mean_b + upper * std_b / n_b_root]
    return c_intv

#def mean_confidence(values, alpha=0.95):
#    return st.t.interval(alpha, len(values)-1, loc=np.mean(values), scale=st.sem(values))

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), st.sem(a)
    h = se * st.t._ppf((1+confidence)/2., n-1)
    return m, m-h, m+h


alpha = 0.5
for measure in ['pearson', 'mse']:
    print('confidence intervalls for %s (order aware, dependency available):' % measure)
    values = res[measure]
    conf_simple = mean_confidence_interval(values, confidence=1-alpha)
    print('%-15s %.1f%% confidence interval: %.4f and %.4f (%.4f) %s naive' % ('ALL', 100 - alpha*100, conf_simple[1],conf_simple[2], conf_simple[2] - conf_simple[1], conf_simple[1] <= conf_simple[0] <= conf_simple[2]))
    c_boot = bootstrap_convidence(values, alpha=alpha)
    print('%-15s %.1f%% confidence interval: %.4f and %.4f (%.4f) %s' % ('ALL', 100 - alpha*100, c_boot[0],c_boot[1], c_boot[1] - c_boot[0], c_boot[0] <= conf_simple[0] <= c_boot[1]))
    for setting in [('NO', 'NO'), ('NO', 'YES'), ('YES', 'NO'), ('YES', 'YES')]:
        values = res.xs(setting)[measure]
        conf_simple = mean_confidence_interval(values, confidence=1-alpha)
        print('%-15s %.1f%% confidence interval: %.4f and %.4f (%.4f) %s naive' % (str(setting), 100 - alpha*100, conf_simple[1],conf_simple[2], conf_simple[2] - conf_simple[1], conf_simple[1] <= conf_simple[0] <= conf_simple[2]))
        c_boot = bootstrap_convidence(values=values, alpha=alpha)
        print('%-15s %.1f%% confidence interval: %.4f and %.4f (%.4f) %s' % (str(setting), 100 - alpha*100, c_boot[0],c_boot[1], c_boot[1] - c_boot[0], c_boot[0] <= conf_simple[0] <= c_boot[1]))
print('finished')  
#print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha*100, lower*100, upper*100))

In [ ]:
ex_values = np.array([110, 112, 106, 90, 96, 118, 108, 114, 107, 90, 85, 84, 113, 105, 90, 104])
print(ex_values.mean())
s = ex_values.std(ddof=1)
print(s*s)
alpha = 0.05
high_p = 1. - (alpha / 2.)
low_p = alpha / 2.
print(low_p)
print(high_p)
low = np.percentile(ex_values, low_p * 100)
high = np.percentile(ex_values, high_p * 100)
print(low)
print(high)

print(mean_confidence_interval(ex_values))

In [ ]:
from sklearn.utils import resample
def bootstrap_5th_percentile(data, n_bootstraps):
    bootstraps = []
    for _ in xrange(n_bootstraps):
        # Sample with replacement from data
        samples = resample(data)
        # Then we take the fifth percentile!
        bootstraps.append(np.percentile(samples, len(data) * 0.1))
    return pd.Series(bootstraps)
values = res.xs(('NO', 'NO'))['pearson']
bootstraps = bootstrap_5th_percentile(values, 10000)
bootstraps.hist()

In [ ]:
print('pearson       mean:  %f' % res[['pearson']].mean().values[0])
print('pearson        std:  %f' % res[['pearson']].std().values[0])
print('pearson_tfidf mean:  %f' % res[['pearson_tfidf']].mean().values[0])
print('mse           mean:  %f' % res[['mse']].mean().values[0])
print('mse            std:  %f' % res[['mse']].std().values[0])
print('mse_tfidf     mean:  %f' % res[['mse_tfidf']].mean().values[0])

res_grouped = {}
res_grouped[('dependency available', 'order aware')] = res.groupby(['dependency available', 'order aware'])
res_grouped['dependency available'] = res.groupby(['dependency available'])
res_grouped['order aware'] = res.groupby(['order aware'])
print(res_grouped[('dependency available', 'order aware')].mean())
print(res_grouped[('dependency available', 'order aware')].std())
print(res_grouped['dependency available'].std())
print(res_grouped['order aware'].std())


#res_grouped[('dependency available', 'order aware')][res_grouped[('dependency available', 'order aware')] == ('NO','NO')]

In [ ]:
# gain
# setting [dependency available, order aware]	pearson	mse	diff pearson	diff mse	% pearson	% mse
# TFIDF	0.6189	0.0823	0.0000	0.0000		
# NO, NO	0.8382	0.0284	0.2193	-0.0539	100.00	100.00
# NO, YES	0.8375	0.0206	0.2186	-0.0617	99.69	114.45
# YES,NO	0.8413	0.0274	0.2224	-0.0549	101.41	101.80
# YES, YES	0.8383	0.0205	0.2194	-0.0618	100.05	114.59

In [ ]:
#res = results_all[['dependency available', 'order aware', 'score_pearson', 'score_mse', 'pearson_tfidf', 'mse_tfidf']]
res_ = {}
res_['pearson'] = results_all[['dependency available', 'order aware','score_pearson']]
res_['mse'] = results_all[['dependency available', 'order aware','score_mse']]
# all settings seperated
pivot = {}
pivot['pearson'] = pd.pivot_table(res_['pearson'], values='score_pearson', index=['crossfold_id', 'dev_file_index'], columns=['dependency available','order aware'])
pivot['mse'] = pd.pivot_table(res_['mse'], values='score_mse', index=['crossfold_id', 'dev_file_index'], columns=['dependency available','order aware'])
# spereated by order aware
pivot_merged = {'pearson':{}, 'mse':{}}
pivot_merged['pearson']['order aware'] = pd.pivot_table(res_['pearson'], values='score_pearson', index=['crossfold_id', 'dev_file_index', 'dependency available'], columns=['order aware'])
pivot_merged['mse']['order aware'] = pd.pivot_table(res_['mse'], values='score_mse', index=['crossfold_id', 'dev_file_index', 'dependency available'], columns=['order aware'])
# spereated by dependency available
#pivot_dep = {}
pivot_merged['pearson']['dependency available'] = pd.pivot_table(res_['pearson'], values='score_pearson', index=['crossfold_id', 'dev_file_index', 'order aware'], columns=['dependency available'])
pivot_merged['mse']['dependency available'] = pd.pivot_table(res_['mse'], values='score_mse', index=['crossfold_id', 'dev_file_index', 'order aware'], columns=['dependency available'])
#pivot['mse'] #['NO', 'NO'] - pivot_mse['NO', 'YES']
#pivot_['pearson']['dependency available']['NO']
pivot['mse'][:10]

In [ ]:
temp = res_['mse'].reset_index().set_index(['dependency available', 'order aware'])
## get best dev_file_index (min score_mse) 
#temp['dev_file_index'].groupby('dev_file_index').min()

## get best score_mse per setting 
## ATTENTION: look up score_mse in full list to get crossfold_id! the crossfold_id displayed here is not the best one
temp[temp['dev_file_index'] == 2].groupby(['dependency available', 'order aware']).min()

In [ ]:
from scipy import stats

for measure in ['mse', 'pearson']:
    for i, s1 in enumerate(settings):
        for s2 in settings[i+1:]:
            #%-15s %.1f
            #if s1 != s2:
            dif = pivot[measure][s2[0], s2[1]].mean() - pivot[measure][s1[0], s1[1]].mean()
            ttest = stats.ttest_ind(pivot[measure][s1[0], s1[1]], pivot[measure][s2[0], s2[1]], equal_var=False)
            print('%-14s vs %-14s %-10s: %10.4f   %.4f  %8.4f' % (str(s1), str(s2), measure, ttest[0], ttest[1], dif))

print()

for measure in measures:
    for parameter in parameters:
        mean_NO = pivot_merged[measure][parameter]['NO'].mean()
        mean_YES = pivot_merged[measure][parameter]['YES'].mean()
        mean_dif = mean_YES - mean_NO
        ttest = stats.ttest_ind(pivot_merged[measure][parameter]['NO'], pivot_merged[measure][parameter]['YES'], equal_var=False)
        print('(merged) %-20s    %-10s  %10.4f   %.4f  (%.4f -> %.4f = %8.4f)' % (parameter, measure, ttest[0], ttest[1], mean_NO, mean_YES, mean_dif))
        #merged = 'order aware'
        #mean_dif = pivot_merged[measure][parameter]['YES'].mean() - pivot_merged[measure][parameter]['NO'].mean()
        #ttest = stats.ttest_ind(pivot_merged[measure][parameter]['NO'], pivot_merged[measure][parameter]['YES'], equal_var=False)
        #print('(merged) %-20s    %-10s  %10.4f   %.4f  %8.4f' % (parameter, measure, ttest[0], ttest[1], mean_dif))

#pivot_merged[measures[0]][parameters[0]]
# interesting results:
#('NO', 'NO')   vs ('NO', 'YES')  pearson   :     1.3194   0.1908
#('NO', 'NO')   vs ('YES', 'YES') pearson   :    -0.2459   0.8064
#('NO', 'YES')  vs ('NO', 'NO')   pearson   :    -1.3194   0.1908
#('NO', 'YES')  vs ('YES', 'YES') mse       :     0.6738   0.5021
#('NO', 'YES')  vs ('YES', 'YES') pearson   :    -1.3679   0.1745
#('YES', 'YES') vs ('NO', 'NO')   pearson   :     0.2459   0.8064
#('YES', 'YES') vs ('NO', 'YES')  mse       :    -0.6738   0.5021
#('YES', 'YES') vs ('NO', 'YES')  pearson   :     1.3679   0.1745

In [ ]:
#res_mse.hist(by='dependency available')
#plt.show
#res_mse.hist(by='order aware')
#plt.show
#res_pearson.hist(by='dependency available')
#plt.show
#res_pearson.hist(by='order aware')
#plt.show
#res_mse.hist()

In [ ]:
#from pandas.plotting import scatter_matrix
#scatter_matrix(pivot_merged['mse']['order aware'], alpha=0.7, figsize=(10, 10), diagonal='kde')

In [ ]:
#scatter_matrix(pivot_pearson, alpha=0.7, figsize=(10, 10), diagonal='kde')

In [ ]:
#'dependency available', 'order aware'
import os
paths = {}
settings = [('NO', 'NO'), ('NO', 'YES'), ('YES', 'NO'), ('YES', 'YES')]
settings = [str(s) for s in settings]
paths[settings[0]] = '/home/arne/ML_local/tf/supervised/log/SA/SICK_RUNS/BEST/AVG_2levels_blanked/6'
paths[settings[1]] = '/home/arne/ML_local/tf/supervised/log/SA/SICK_RUNS/BEST/LSTM_2levels_blanked/9'
paths[settings[2]] = '/home/arne/ML_local/tf/supervised/log/SA/SICK_RUNS/BEST/AVG_2levels/8'
paths[settings[3]] = '/home/arne/ML_local/tf/supervised/log/SA/SICK_RUNS/BEST/LSTM_2levels/2'
sims_predicted = {}
sims_gold_ = {}
errors_ = {}
sims_all = {}
for setting in settings:
    sims_predicted[setting] = np.load(os.path.join(paths[setting], 'sims.np'))
    sims_all[setting] = sims_predicted[setting]
    sims_gold_[setting] = np.load(os.path.join(paths[setting], 'sims_gold.np'))
    errors_[setting] =sims_predicted[setting]-sims_gold_[setting]
sims_gold = sims_gold_[settings[0]]
sims_predicted['sims_gold'] = sims_gold
sims_predicted = pd.DataFrame(sims_predicted)
sims_predicted


In [ ]:
#errors_df = pd.DataFrame(errors_)
errors_orderawareNO = np.concatenate([errors_[settings[0]], errors_[settings[2]]])
errors_orderawareYES = np.concatenate([errors_[settings[1]], errors_[settings[3]]])
errors_dependencyNO = np.concatenate([errors_[settings[0]], errors_[settings[1]]])
errors_dependencyYES = np.concatenate([errors_[settings[2]], errors_[settings[3]]])
print('errors_orderaware (NO -> YES): %.3f -> %.3f (%.4f)' % (errors_orderawareNO.mean(), errors_orderawareYES.mean(), errors_orderawareYES.mean() - errors_orderawareNO.mean()))
print(stats.ttest_ind(errors_orderawareNO, errors_orderawareYES))
print('errors_dependency (NO -> YES): %.3f -> %.3f (%.4f)' % (errors_dependencyNO.mean(), errors_dependencyYES.mean(), errors_dependencyYES.mean() - errors_dependencyNO.mean()))
print(stats.ttest_ind(errors_dependencyNO, errors_dependencyYES))

In [ ]:
sims_gold_merged = sims_predicted.sort_values('sims_gold').groupby('sims_gold').mean().reset_index()[['sims_gold']]
plot_temp = sims_gold_merged.plot(color='k')
#plot_temp = sims_predicted.sort_values('sims_gold').reset_index()[['sims_gold']].plot(color='k')
#plot_temp.get_yaxis().set_visible(True)
plot_temp.grid('on', which='major', axis='y' )
plot_temp.grid('on', which='minor', axis='y', linestyle='-')
#plot_temp.tick_params(axis='x',which='minor',bottom='off')
#plot_temp.tick_params(axis='y',which='minor',bottom='off')
plot_temp.minorticks_on()

In [ ]:
temp = sims_predicted.sort_values('sims_gold').reset_index().drop('index', 1).reset_index()
figsize = (13, 10)
fig_predictions_bp, axs = plt.subplots(2,2, figsize=figsize)
for ax in np.reshape(axs, 4):
    temp.plot.scatter(x='index', y=settings[0], alpha=.3, s=5, ax=ax, label='prediction')
    temp[['sims_gold']].plot(ax=ax, color='black')
    ax.lines[0].set_label('gold')
    ax.set_ylabel('similarity score')
    ax.set_xlabel('')
    ax.get_xaxis().set_visible(False)
    ax.legend()


In [ ]:
temp = sims_predicted.sort_values('sims_gold').reset_index().drop('index', 1).reset_index()
figsize = (13, 10)
fig_sorted_errors_sep, axs = plt.subplots(2,2, figsize=figsize)

for i, ax in enumerate(np.reshape(axs, 4)):
    x = temp[['sims_gold']].plot(ax=ax, color='black')
    line = x.lines[0]
    _errs = pd.DataFrame()
    _errs['neg'] = temp.apply(lambda row: min(row[settings[i]] - row['sims_gold'], 0), axis=1)
    _errs['pos'] = temp.apply(lambda row: max(row[settings[i]] - row['sims_gold'], 0), axis=1)
    _errs_neg = np.minimum(temp[settings[i]].values - temp['sims_gold'].values, 0.0)
    _errs_pos = np.maximum(temp[settings[i]].values - temp['sims_gold'].values, 0.0)
    #errs_neg =  max(temp[settings[i]] - temp['sims_gold'], 0)
    ax.errorbar(line.get_xdata(), line.get_ydata(), yerr=[_errs_neg * -1., _errs_pos], label='predictions', capsize=0, alpha=.3)
    line.set_label('gold')
    ax.set_ylabel('similarity score')
    ax.set_xlabel('')
    ax.get_xaxis().set_visible(False)
    ax.set_title(settings[i])#, y=y_title)
    ax.legend(loc="lower right")

print_images({'fig_sorted_errors_sep':fig_sorted_errors_sep})

In [ ]:
figsize = (13, 10)
fig_predictions_bp, axs = plt.subplots(2,2, figsize=figsize)
for ax in np.reshape(axs, 4):
    p = sims_gold_merged.plot(ax=ax, color='k')
    p.legend(loc="lower right")

flierprops = dict(marker='x', markerfacecolor='grey', markersize=4, linestyle='none', alpha=0.6)

boxplot_axes = sims_predicted.sort_values('sims_gold').boxplot(by='sims_gold', figsize=figsize, showfliers=True, flierprops=flierprops, showcaps=False, widths=1., ax=axs)
#print(len(boxplot_axes))
fig_predictions_bp.suptitle('')

for bp in boxplot_axes:
    #print('X')
    #print(bp)
    #gold_plot = sims_gold_merged.plot(use_index=False, ax=bp, color='k')
    #gold_plot.legend(loc="lower right")
    
    bp.get_xaxis().set_visible(False)
    bp.set_ylabel('similarity score')
    bp.grid('on', which='major', axis='y' )
    bp.grid('on', which='minor', axis='y' )
    bp.minorticks_on()

print_images({'fig_predictions_bp':fig_predictions_bp})

In [ ]:
#sims_predicted.index.names = ['index']
#sims_predicted = sims_predicted.reset_index()
sims_predicted_grouped = sims_predicted.groupby("sims_gold").agg([min, max, np.mean]).reset_index()
sims_predicted['index'] = range(len(sims_predicted))
sims_predicted_grouped['indices'] = sims_predicted.groupby("sims_gold")["index"].apply(list).reset_index()['index']
sims_predicted = sims_predicted.drop(['index'], axis=1)

def max_dif(_min, _max, _gold):
    if abs(_min - _gold) > abs(_max - _gold):
        return _min
    else:
        return _max
    

#for s in settings:
    #sims_predicted_grouped[s, 'max_dif'] = sims_predicted_grouped.apply(lambda row: row[s, 'min'] if row[s, 'min'] < row[s, 'mean'] else row[s, 'max'])
    #sims_predicted_grouped[s, 'max_dif'] = sims_predicted_grouped[s, 'mean'] #sims_predicted_grouped.apply(lambda row: row[s, 'min'])
    #sims_predicted_grouped[s, 'max_dif'] = sims_predicted_grouped.apply(lambda row: max_dif(row[s, 'min'], row[s, 'max'], row['sims_gold']), axis=1)
    #sims_predicted_grouped[s, 'max_dif'] = sims_predicted_grouped[[(s, 'min'), (s, 'max')]].sub(sims_predicted_grouped[['sims_gold']]).max(axis=1)
#    sims_predicted_grouped[s, 'min_dif'] = sims_predicted_grouped.apply(lambda row: abs(row[s, 'min'] - row['sims_gold']), axis=1)
#    sims_predicted_grouped[s, 'max_dif'] = sims_predicted_grouped.apply(lambda row: abs(row[s, 'max'] - row['sims_gold']), axis=1)
    
#df.groupby('a')['b'].apply(list)
#sims_predicted.groupby("sims_gold")["index"].apply(list).reset_index()['index']
#sims_predicted.groupby("sims_gold").agg([np.mean])

sims_errors_grouped = pd.DataFrame()
sims_errors_grouped['sims_gold'] = sims_predicted_grouped['sims_gold']
for s in settings:
    for agg in ['min', 'max', 'mean']:
        sims_errors_grouped['error %s %s' % (agg, s)] = sims_predicted_grouped[(s,agg)] - sims_predicted_grouped['sims_gold']
sims_errors_grouped['indices'] = sims_predicted_grouped['indices']

#sims_errors_grouped

sims_errors_grouped
#sims_predicted_grouped[[(settings[0], 'min')]]#.sub(sims_predicted_grouped[['sims_gold']])

In [ ]:
#[['sims_gold', ("('NO', 'NO')", 'mean')]]

In [ ]:

figsize = (13, 10)
images_2 = {}
#ax = plt.figure(figsize=figsize)
#plot = sims_predicted_grouped[['sims_gold', ("('NO', 'NO')", 'max_dif')]].plot(use_index=False, alpha=0.7, figsize=figsize)
## all
idx = pd.IndexSlice
means_and_gold = sims_predicted_grouped.sort_index().sort_index(axis=1).loc[:, idx[settings+['sims_gold'],['mean', '']]]
#means_and_gold = sims_predicted_grouped.sort_index().sort_index(axis=1).loc[:, idx[["('YES', 'NO')", "('YES', 'YES')", 'sims_gold'],['mean', '']]]

plot = means_and_gold.plot(use_index=False, alpha=0.7, figsize=figsize)
ax = plt.gca()
ax.legend(loc="lower right")
#fig_predictions_sep = plot.get_figure()
#images_2['fig_predictions_mean_all'] = plot.get_figure()
print_images({'fig_predictions_mean_togeth': plot.get_figure()})

fig_predictions_area_sep, axs = plt.subplots(2,2, figsize=figsize)
#sims_predicted.sort_values(by=["sims_gold"], ascending=True)[["sims_gold", "('NO', 'NO')"]].groupby("sims_gold").plot(use_index=False, alpha=0.7, figsize=figsize, ax=axs[0][0])
for i, s in enumerate(settings):
    x = i / 2
    y = i % 2
    
    ax = sims_predicted_grouped[["sims_gold", s]].plot(use_index=False, alpha=0.7, figsize=figsize, ax=axs[x][y])
    ax.fill_between(ax.lines[0].get_xdata(), ax.lines[1].get_ydata(), ax.lines[2].get_ydata(), label='predictions')
    # configure gold_sims ...
    ax.lines[0].set_color("black")
    ax.lines[0].set_label('gold similarities')
    # ... and min ...
    ax.lines[1].set_label('')
    ax.lines[1].set_color("navy")
    # ... and max ...
    ax.lines[2].set_color("navy")
    ax.lines[2].set_label('')
    # ... and mean ...
    ax.lines[3].set_linestyle('None')
    ax.lines[3].set_label('')
    ax.set_title(s)
    #ax.fill_between(ax.lines[0].get_xdata(), ax.lines[0].get_ydata(), ax.lines[1].get_ydata())
    axs[x][y].legend(loc="lower right")
    
print_images({'fig_predictions_area_sep':fig_predictions_area_sep})

fig_predictions_mean_sep, axs = plt.subplots(2,2, figsize=figsize)
#sims_predicted.sort_values(by=["sims_gold"], ascending=True)[["sims_gold", "('NO', 'NO')"]].groupby("sims_gold").plot(use_index=False, alpha=0.7, figsize=figsize, ax=axs[0][0])
for i, s in enumerate(settings):
    x = i / 2
    y = i % 2
    
    ax = sims_predicted_grouped[["sims_gold", s]].plot(use_index=False, alpha=0.7, figsize=figsize, ax=axs[x][y])
    #ax.fill_between(ax.lines[0].get_xdata(), ax.lines[1].get_ydata(), ax.lines[2].get_ydata(), label='predictions')
    # configure gold_sims ...
    ax.lines[0].set_linestyle('-.')
    ax.lines[0].set_color("black")
    ax.lines[0].set_label('gold similarities')
    # ... and min ...
    ax.lines[1].set_linestyle('None')
    ax.lines[1].set_label('')
    #ax.lines[1].set_color("navy")
    # ... and max ...
    ax.lines[2].set_linestyle('None')
    ax.lines[2].set_label('')
    #ax.lines[2].set_color("navy")
    # ... and mean ...
    #ax.lines[3].set_linestyle('None')
    ax.lines[3].set_color("blue")
    ax.lines[3].set_label('mean predictions')
    ax.set_title(s)
    #ax.fill_between(ax.lines[0].get_xdata(), ax.lines[0].get_ydata(), ax.lines[1].get_ydata())
    axs[x][y].legend(loc="lower right")

print_images({'fig_predictions_mean_sep':fig_predictions_mean_sep})

In [ ]:
largest_pairs_grouped_neg = {}
largest_pairs_grouped_pos = {}
for s in settings:
    l = list(sims_errors_grouped.sort_values(by="error mean "+s, ascending=True)['indices'][:10])
    largest_pairs_grouped_neg[s]=sorted([item for sublist in l for item in sublist])
    l = list(sims_errors_grouped.sort_values(by="error mean "+s, ascending=False)['indices'][:10])
    largest_pairs_grouped_pos[s]=sorted([item for sublist in l for item in sublist])
#pd.DataFrame(largest_pairs_grouped_neg)
pd.DataFrame(largest_pairs_grouped_pos)

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

errors_top100 = {}
errors_top10 = {}
errors = pd.DataFrame(errors_)
errors_abs = pd.DataFrame()
errors['sentence_A'] = sick_test['sentence_A']
errors['sentence_B'] = sick_test['sentence_B']
errors['score_gold'] = sick_test['relatedness_score'].sub(1.).div(4.)
errors['score_jaccard'] = sick_test['sim_jaccard']
errors['score_len'] = sick_test['sim_len']
errors['len_total'] = sick_test['len_total']

#errors_dif = {}
for i, s1 in enumerate(settings):
    #all_sorted = errors.sort_values(by=[s1], ascending=False)
    #errors_top100[s1] = all_sorted[:100]
    #errors_top10[s1] = all_sorted[:10]
    errors['%s abs' % s1] = errors[s1].abs()
    for s2 in settings[i+1:]:
        errors['%s abs' % s2] = errors[s2].abs()
        #errors_dif[s1,s2] = errors[s2]-errors[s1]
        errors['%s vs %s' % (str(s1),str(s2))] = errors[s2]-errors[s1]
        errors['%s vs %s abs' % (str(s1),str(s2))] = (errors[s2]-errors[s1]).abs()
        errors['%s abs vs %s abs' % (str(s1),str(s2))] = errors['%s abs' % s2]-errors['%s abs' % s1]
        errors['%s abs vs %s abs abs' % (str(s1),str(s2))] = (errors['%s abs' % s2]-errors['%s abs' % s1]).abs()

# absolute error values
#errors_abs = pd.concat([errors.select_dtypes(exclude=['number']), errors.select_dtypes(include=['number']).apply(np.abs)], axis=1)
#errors_abs = errors
#for column in errors.select_dtypes(include=['number']):
#    errors_abs['%s abs' % column] = errors[column].apply(np.abs)
#errors_abs

#errors_top100[('NO', 'YES')]['score_gold'].mean()
#errors_top10[('NO', 'YES')]#[('NO', 'YES'), 'sentence_A', 'sentence_B', 'score_gold']
#errors_dif.keys() # (('NO', 'YES'), ('YES', 'YES')) #('NO', 'NO') vs ('NO', 'YES')
#errors_dif[(('NO', 'YES'), ('YES', 'YES'))]

def top_n_stats(s1, s2, n=10):
    result = pd.DataFrame()
    # compare performance
    # which is better?
    temp = errors.sort_values(by=["%s abs vs %s abs" % (s1, s2)], ascending=True)[:n]
    temp['type'] = "%s better" % s2
    result = result.append(temp)
    temp = errors.sort_values(by=["%s abs vs %s abs" % (s1, s2)], ascending=False)[:n]
    temp['type'] = "%s better" % s1
    result = result.append(temp)
    # similar or not?
    temp = errors.sort_values(by=["%s abs vs %s abs abs" % (s1, s2)], ascending=True)[:n]
    temp['type'] = "error dif min"
    result = result.append(temp)
    temp = errors.sort_values(by=["%s abs vs %s abs abs" % (s1, s2)], ascending=False)[:n]
    temp['type'] = "error dif max"
    result = result.append(temp)
    # individual over/under rating
    temp = errors.sort_values(by=["%s" % s1], ascending=True)[:n]
    temp['type'] = "%s underates" % s1
    result = result.append(temp)
    temp = errors.sort_values(by=["%s" % s2], ascending=True)[:n]
    temp['type'] = "%s underates" % s2
    result = result.append(temp)
    temp = errors.sort_values(by=["%s" % s1], ascending=False)[:n]
    temp['type'] = "%s overrates" % s1
    result = result.append(temp)
    temp = errors.sort_values(by=["%s" % s2], ascending=False)[:n]
    temp['type'] = "%s overrates" % s2
    result = result.append(temp)
    
    # filter columns
    remove_cols = []
    for col in list(result):
        for rem_s in settings:
            if rem_s not in [s1, s2] and rem_s in col:
                remove_cols.append(col)
    col_names = set(result) - set(remove_cols)
    result = result[sorted(list(col_names))]
    
    # store index
    result = result.reset_index()
    # move column 'type' to front
    cols = list(result)
    cols.insert(0, cols.pop(cols.index('type')))
    result = result.reindex(columns= cols)
    return result
    

top_n_stats("('NO', 'NO')", "('NO', 'YES')")

# order aware rocks (dep edge off) 
#errors.sort_values(by=["('NO', 'NO') vs ('NO', 'YES')"], ascending=True)[:10]

# order aware fails (dep edge off) 
#errors.sort_values(by=["('NO', 'NO') vs ('NO', 'YES')"], ascending=False)[:10]

# order aware rocks (dep edge on) 
#errors.sort_values(by=["('YES', 'NO') vs ('YES', 'YES') abs"], ascending=True)[:10]

# order aware fails (dep edge on) 
#errors.sort_values(by=["('YES', 'NO') vs ('YES', 'YES') abs"], ascending=False)[:10]


# dep edge rocks (order aware off) 
#errors.sort_values(by=["('NO', 'NO') vs ('YES', 'NO')"], ascending=True)[:10]

# dep edge fails (order aware off) 
#errors.sort_values(by=["('NO', 'NO') vs ('YES', 'NO')"], ascending=False)[:10]

# dep edge rocks (order aware on) 
#errors.sort_values(by=["('NO', 'YES') vs ('YES', 'YES')"], ascending=True)[:10]

# dep edge fails (order aware on) 
#errors.sort_values(by=["('NO', 'YES') vs ('YES', 'YES')"], ascending=False)[:10]

# just 
#errors[settings].sort_values(by="('NO', 'NO')", ascending=True)[:10]


In [ ]:
errors.corr()

In [ ]:
# sorted indices by error
error_indices_sorted = pd.DataFrame()
for s in settings:
    error_indices_sorted[s]=errors[settings].abs().sort_values(by=s, ascending=False).index
error_indices_sorted

In [ ]:
#error_indices_sorted.sort_values("('NO', 'NO')").index - error_indices_sorted.sort_values("('NO', 'YES')").index

In [ ]:
# check overlop of indices between settings if taking the n smallest (or largest errors)
overlaps = []
#max_n = len(errors)
max_n = 200
for trunc_size in range(1, max_n):
    overlaps.append({})
    for i, s1 in enumerate(settings):
        s1_indices_set = set(error_indices_sorted[s1][:trunc_size].tolist())
        for s2 in settings[i+1:]:
            s2_indices_set = set(error_indices_sorted[s2][:trunc_size].tolist())
            union = s1_indices_set | s2_indices_set
            intersec = s1_indices_set & s2_indices_set
            ## revert (take last / smallest errors)
            #se = set(error_indices_sorted[s1][::-1][:trunc_size].tolist())
            #se.update(error_indices_sorted[s2][::-1][:trunc_size].tolist())
            #overlaps[-1]['%s vs %s' % (s1, s2)] = (2*trunc_size - len(se)) / float(trunc_size)
            overlaps[-1]['%s vs %s' % (s1, s2)] = len(intersec) / float(len(union))
            #print('%s vs %s' % (s1, s2, 2*trunc_size - len(se)))#, se)

pd.DataFrame(overlaps)#.plot(figsize=(15,10))

In [ ]:
error_indices_ranked_dif = pd.DataFrame()
#error_indices_ranked_dif = {}
for i, s1 in enumerate(settings):
    #error_indices_ranked[s1]=sorted(errors[settings].abs().sort_values(by=s1, ascending=True).index.tolist())
    for s2 in settings[i+1:]:
        #error_indices_ranked[s2]=sorted(errors[settings].abs().sort_values(by=s2, ascending=True).index.tolist())
        #se = set(error_indices_neg[s1])
        #se.update(error_indices_neg[s2])
        error_indices_ranked_dif['%s vs %s' % (s1, s2)] = error_indices_sorted.sort_values(s1).index - error_indices_sorted.sort_values(s2).index
        #print('%s vs %s' % (s1, s2, 2*trunc_size - len(se)))#, se)

#error_indices_ranked_dif.hist(figsize=(15,10), alpha=.5, bins=100)
#error_indices_ranked_dif.plot.hist(figsize=(15,10), alpha=.5, bins=200)
#error_indices_ranked_dif#.plot(figsize=(15,10), alpha=.5)
error_indices_ranked_dif

#fig, ax = plt.subplots(1,1,figsize=(15,10))
#for column in error_indices_ranked_dif:
#    error_indices_ranked_dif[column].sort_values(ascending=False).plot(use_index=False, alpha=.5, figsize=(15,10))
#    #break



In [ ]:
#for s in settings:
#    print('%-14s %.4f   %.4f' % (str(s), errors_top10[s]['score_gold'].mean(), errors_top100[s]['score_gold'].std()))
    #print(errors_top100[s]['score_gold'].std())

In [ ]:
# biggest deviations (negative)
#errors.iloc[[2389, 2539, 732, 1569]]